Übung: Verteilung 
-----------------

In dieser Übung erstellen wir mehrere Pods ab dem gleichen Image mit jeweils einem ReplicaSet, Deployment und Service.

Das passiert in einer eigenen Namespace um die Resultate gezielt Darstellen zu können:

In [40]:
! kubectl create namespace rs

namespace/rs created


Erzeugen eines Deployments, hier der das Beispiel von Docker mit einem Web Server welche die aktuelle IP-Adresse ausgibt.

In [41]:
! kubectl run apache --image=dockercloud/hello-world --namespace rs

kubectl run --generator=deployment/apps.v1beta1 is DEPRECATED and will be removed in a future version. Use kubectl create instead.
deployment.apps/apache created


Ausgabe der Erzeugten Ergebnisse und die YAML Datei welche den Erzeugten Ressourcen beschreibt.

Ab `spec.containers` kommt erst der Pod.

In [42]:
! kubectl get pod,deployment,replicaset,service --namespace rs

NAME                          READY   STATUS    RESTARTS   AGE
pod/apache-7d79f75fd4-z99n6   1/1     Running   0          3s

NAME                           DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deployment.extensions/apache   1         1         1            1           3s

NAME                                      DESIRED   CURRENT   READY   AGE
replicaset.extensions/apache-7d79f75fd4   1         1         1       3s


In [43]:
! kubectl get deployment apache -o yaml --namespace rs | head -39

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  annotations:
    deployment.kubernetes.io/revision: "1"
  creationTimestamp: 2018-10-25T11:42:49Z
  generation: 1
  labels:
    run: apache
  name: apache
  namespace: rs
  resourceVersion: "843793"
  selfLink: /apis/extensions/v1beta1/namespaces/rs/deployments/apache
  uid: 19761b9f-d84b-11e8-9739-02fba84c29d0
spec:
  progressDeadlineSeconds: 600
  replicas: 1
  revisionHistoryLimit: 2
  selector:
    matchLabels:
      run: apache
  strategy:
    rollingUpdate:
      maxSurge: 25%
      maxUnavailable: 25%
    type: RollingUpdate
  template:
    metadata:
      creationTimestamp: null
      labels:
        run: apache
    spec:
      containers:
      - image: dockercloud/hello-world
        imagePullPolicy: Always
        name: apache
        resources: {}
        terminationMessagePath: /dev/termination-log
        terminationMessagePolicy: File


Um die ReplicaSet Funktionalität zu Demonstrieren, setzen wir die Anzahl der laufenden Pods auf 3 und schauen uns das Ergebnis an

In [44]:
! kubectl --namespace rs scale --replicas=3 deployment/apache  
! kubectl get pod,deployment,replicaset,service --namespace rs

deployment.extensions/apache scaled
NAME                          READY   STATUS              RESTARTS   AGE
pod/apache-7d79f75fd4-xbbgl   0/1     ContainerCreating   0          1s
pod/apache-7d79f75fd4-xcfz6   0/1     ContainerCreating   0          1s
pod/apache-7d79f75fd4-z99n6   1/1     Running             0          20s

NAME                           DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deployment.extensions/apache   3         3         3            1           20s

NAME                                      DESIRED   CURRENT   READY   AGE
replicaset.extensions/apache-7d79f75fd4   3         3         1       20s


Zu dem **Deployment** `apache` Erzeugen wir einen Service. Dadurch wird der Web Server von aussen sichtbar.

Der Port 80 wird von Kubernetes automatisch auf den nächsten freien Port gemappt.

Das Ergebnis schauen wir uns wieder an.

In [45]:
! kubectl expose deployment/apache --type="LoadBalancer" --port 80 --namespace rs
! kubectl get pod,deployment,replicaset,service --namespace rs

service/apache exposed
NAME                          READY   STATUS    RESTARTS   AGE
pod/apache-7d79f75fd4-xbbgl   1/1     Running   0          6s
pod/apache-7d79f75fd4-xcfz6   1/1     Running   0          6s
pod/apache-7d79f75fd4-z99n6   1/1     Running   0          25s

NAME                           DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deployment.extensions/apache   3         3         3            3           25s

NAME                                      DESIRED   CURRENT   READY   AGE
replicaset.extensions/apache-7d79f75fd4   3         3         3       25s

NAME             TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
service/apache   LoadBalancer   10.98.81.103   <pending>     80:32441/TCP   1s


Da wir keinen LoadBalancer haben müssen wir mit einem kleinen Shellscript selber die IP des Clusters und der gemappte Port als URL aufbereiten.

Diese Shellscript ist im Script `startsvc` hinterlegt.

In [46]:
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service --namespace rs apache -o=jsonpath='{ .spec.ports[0].nodePort }')/"


http://192.168.178.200:32441

Wird der obige Link in mehreren Browsern geöffnet sollte jeweils ein anderer Hostname erscheinen. 

**Erklärung**: Kubernetes verteilt die Anfragen auf die laufenden Pods.

Zum Aufräumen genügt es den Namespace zu löschen

In [ ]:
! kubectl delete namespace rs